# 2. Implementation

## 2.1 State Space Search

### Helper methods

In [10]:
import copy
import time

def isGoalState(puzzle):
    if puzzle[2][4] == "A" and puzzle[2][5] == "A":
        return True
    return False

def runValetServiceCheckup(newPuzzle):
    if newPuzzle[2][5] != "A" and newPuzzle[2][5] == newPuzzle[2][4]:
        #car is horizontal and is ready for valet
        carType = newPuzzle[2][5]
        newCol = 5
        while(newCol>=0):
            if newPuzzle[2][newCol] == carType:
                newPuzzle[2][newCol] = "."
            else:
                break
            newCol -= 1
    return newPuzzle

def findHorizontalMovesOfCar(solutionPath, puzzle, y, startX, endX, fuelLevels, heuristicFnct):
    newNodes = []
    carLength = endX - startX + 1
    carType = puzzle[y][startX]

    for posX in reversed(range(startX)):
        #move car left
        if puzzle[y][posX] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = startX - posX
        if displacement <= newFuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            carPartsToMove = carLength
            for newIndexX in range(posX, endX+1):
                if carPartsToMove > 0:
                    newPuzzle[y][newIndexX] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[y][newIndexX] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newSolutionPath.append({
                "carType":carType,
                "direction":"right",
                "fuelLevels":newFuelLevels,
                "puzzle":newPuzzle
            })
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels,
                "stringConfig":puzzleToStringConfig(newPuzzle),
            }
            newNode["fOfN"] = heuristicFnct(newNode)
            newNodes.append(newNode)
        else:
            break

    for posX in range(endX+1, 6):
        #move car right
        if puzzle[y][posX] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = posX - endX
        if displacement <= fuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            carPartsToMove = carLength
            for newIndexX in reversed(range(startX, posX+1)):
                if carPartsToMove > 0:
                    newPuzzle[y][newIndexX] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[y][newIndexX] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newSolutionPath.append({
                "carType":carType,
                "direction":"right",
                "fuelLevels":newFuelLevels,
                "puzzle":newPuzzle
            })
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels,
                "stringConfig":puzzleToStringConfig(newPuzzle),
            }
            newNode["fOfN"] = heuristicFnct(newNode)
            newNodes.append(newNode)
        else:
            break

    return newNodes

def findVerticalMovesOfCar(solutionPath, puzzle, x, startY, endY, fuelLevels, heuristicFnct):
    newNodes = []
    carLength = endY - startY + 1
    carType = puzzle[startY][x]

    for posY in reversed(range(startY)):
        #move car up
        if puzzle[posY][x] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = startY - posY
        if displacement <= fuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            carPartsToMove = carLength
            for newIndexY in range(posY, endY+1):
                if carPartsToMove > 0:
                    newPuzzle[newIndexY][x] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[newIndexY][x] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newSolutionPath.append({
                "carType":carType,
                "direction":"right",
                "fuelLevels":newFuelLevels,
                "puzzle":newPuzzle
            })
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels,
                "stringConfig":puzzleToStringConfig(newPuzzle),
            }
            newNode["fOfN"] = heuristicFnct(newNode)
            newNodes.append(newNode)
        else:
            break

    for posY in range(endY+1, 6):
        #move car down
        if puzzle[posY][x] != ".":
            break

        newPuzzle = copy.deepcopy(puzzle)
        newFuelLevels = copy.deepcopy(fuelLevels)
        newSolutionPath = copy.deepcopy(solutionPath)

        displacement = posY - endY
        if displacement <= fuelLevels[carType]:
            newFuelLevels[carType] = newFuelLevels[carType] - displacement
            carPartsToMove = carLength
            for newIndexY in reversed(range(startY, posY+1)):
                if carPartsToMove > 0:
                    newPuzzle[newIndexY][x] = carType
                    carPartsToMove -= 1
                else:
                    newPuzzle[newIndexY][x] = "."

            newPuzzle = runValetServiceCheckup(newPuzzle)
            newSolutionPath.append({
                "carType":carType,
                "direction":"right",
                "fuelLevels":newFuelLevels,
                "puzzle":newPuzzle
            })
            newNode = {
                "puzzle":newPuzzle,
                "solutionPath":newSolutionPath,
                "fuelLevels":newFuelLevels,
                "stringConfig":puzzleToStringConfig(newPuzzle),
            }
            newNode["fOfN"] = heuristicFnct(newNode)
            newNodes.append(newNode)
        else:
            break
    return newNodes

def findSuccessors(node, heuristicFnct):
    newNode = copy.deepcopy(node)
    solutionPath = newNode["solutionPath"]
    successorsList = []
    puzzle = newNode["puzzle"]
    movedCars = []
    fuelLevels = newNode["fuelLevels"]
    for row in range(len(puzzle)):
        for col in range(len(puzzle[row])):
            if (puzzle[row][col] != ".") and (puzzle[row][col] not in movedCars):
                movedCars.append(puzzle[row][col])
                if (col+1 != 6) and (puzzle[row][col] == puzzle[row][col+1]):
                    #horizontal car
                    endX = col
                    for newX in range(col, len(puzzle[row])):
                        if puzzle[row][newX] != puzzle[row][col]:
                            break
                        else:
                            endX = newX
                    horizontalMoves = findHorizontalMovesOfCar(solutionPath, puzzle, row, col, endX, fuelLevels, heuristicFnct)
                    successorsList = successorsList + horizontalMoves
                elif (row+1 != 6) and (puzzle[row][col] == puzzle[row+1][col]):
                    #vertical car
                    endY = row
                    for newY in range(row, len(puzzle)):
                        if puzzle[newY][col] != puzzle[row][col]:
                            break
                        else:
                            endY = newY
                    verticalMoves = findVerticalMovesOfCar(solutionPath, puzzle, col, row, endY, fuelLevels, heuristicFnct)
                    successorsList = successorsList + verticalMoves
    return successorsList

def arePuzzlesEqual(puzzle1, puzzle2):
    for row in range(len(puzzle1)):
        for col in range(len(puzzle1[row])):
            if puzzle1[row][col] != puzzle2[row][col]:
                return False
    return True

def printCutePuzzle(puzzle):
    for line in puzzle:
        str = ""
        for elem in line:
            str += elem + " "
        print(str+"\n")
    print("\n--------------------------\n")

def puzzleToStringConfig(puzzle):
    stringConfig = "".join(str(elem) for row in puzzle for elem in row)
    return stringConfig

def solver(puzzle, fuelLevels, heuristic, algorithm):
    startTime = time.time()
    initialState = {
        "puzzle":puzzle,
        "solutionPath":[],
        "fuelLevels":fuelLevels,
        "stringConfig":puzzleToStringConfig(puzzle)
    }
    initialState["fOfN"] = heuristic(initialState)
    openList = [initialState]
    closedList = []

    shadowClosedList = {}
    shadowOpenList = {
        initialState["stringConfig"]:1
    }

    nbSimilar = 0
    nbSimilar2 = 0

    while(len(openList) > 0):
        firstNode = openList.pop(0)
        del shadowOpenList[firstNode["stringConfig"]]
        if isGoalState(firstNode["puzzle"]):
            endTime = time.time()
            return {
                "result": "ok",
                "node": firstNode,
                "runtime": endTime - startTime,
                "lengthSearchPaths": len(closedList),
                "closedList":closedList
            }
        else:
            closedList.append(firstNode)
            shadowClosedList[firstNode["stringConfig"]] = 1
            successors = findSuccessors(firstNode, heuristic)
            for successor in successors:
                if successor["stringConfig"] in shadowClosedList:
                    nbSimilar+=1
                    if algorithm == "a":
                        for i in range(len(closedList)):
                            if successor["stringConfig"] == closedList[i]["stringConfig"]:
                                if successor["fOfN"] < closedList[i]["fOfN"]:
                                    openList.append(successor)
                                    shadowOpenList[successor["stringConfig"]] = 1
                                    del shadowClosedList[successor["stringConfig"]]
                                    closedList.pop(i)
                                break
                else:
                    if successor["stringConfig"] in shadowOpenList:
                        for i in range(len(openList)):
                            if arePuzzlesEqual(successor["puzzle"], openList[i]["puzzle"]):
                                nbSimilar2 += 1
                                if successor["fOfN"] < openList[i]["fOfN"]:
                                    openList[i] = successor
                                break
                    else:
                        openList.append(successor)
                        shadowOpenList[successor["stringConfig"]] = 1
            openList.sort(key=lambda x: x["fOfN"], reverse=False)

        print("\rLength closedList: "+str(len(closedList))+"; Length open list: "+str(len(openList))+"; Closed similar:"+str(nbSimilar)+"; Open similar:"+str(nbSimilar2), end="");

    endTime = time.time()
    return {
                "result": "no solution",
                "runtime": endTime - startTime,
                "lengthSearchPaths": len(closedList),
                "closedList":closedList
            }

#### Uniform Cost Search heuristic

In [11]:
def hUcs(node):
    return len(node["solutionPath"])


#### GBFS heuristics

In [12]:
def h1Gbfs(node):
    puzzle = node["puzzle"]
    isAmbulanceFound = False
    uniqueCarTypes = []
    for col in range(0, 6):
        value = puzzle[2][col]
        if isAmbulanceFound == True:
            if (value != "A") and (value != ".") and (value not in uniqueCarTypes):
                uniqueCarTypes.append(value)
        else:
            if value == "A":
                isAmbulanceFound = True
    return len(uniqueCarTypes)

def h2Gbfs(node):
    puzzle = node["puzzle"]
    isAmbulanceFound = False
    count = 0
    for col in range(0, 6):
        value = puzzle[2][col]
        if isAmbulanceFound == True:
            if (value != "A") and (value != "."):
                count += 1
        else:
            if value == "A":
                isAmbulanceFound = True
    return count

def h3Gbfs(node):
    return h1Gbfs(node) * 3

def h4Gbfs(node):
    #like h1, but count car twice if it is itself blocked
    puzzle = node["puzzle"]
    isAmbulanceFound = False
    uniqueCarTypes = []
    count = 0
    for col in range(0, 6):
        value = puzzle[2][col]
        if isAmbulanceFound == True:
            #search for cars to the right of ambulance
            if (value != "A") and (value != ".") and (value not in uniqueCarTypes):
                uniqueCarTypes.append(value)
                count += 1
                if col < 5 and puzzle[2][col+1] == value:
                    #car is horizontal
                    if puzzle[2][col-1] != ".":
                        #is blocked left
                        newCol = col
                        isBlocked = False
                        while(newCol <= 5):
                            if puzzle[2][newCol] == ".":
                                break
                            elif puzzle[2][newCol] != value:
                                isBlocked = True
                                #is also blocked right
                                break
                            newCol+=1
                        if isBlocked:
                            count+=1
                elif puzzle[1][col] == value or puzzle[3][col] == value:
                    #car is vertical
                    newRowUp = 1
                    isBlockedUp = False
                    while(newRowUp >= 0):
                        if puzzle[newRowUp][col] == ".":
                            break
                        elif puzzle[newRowUp][col] != value:
                            isBlockedUp = True
                            #is blocked up
                            break
                        newRowUp -= 1
                    if isBlockedUp:
                        newRowDown = 3
                        isBlockedDown = False
                        while(newRowDown<=5):
                            if puzzle[newRowDown][col] == ".":
                                break
                            elif puzzle[newRowDown][col] != value:
                                isBlockedDown = True
                                #is blocked down
                                break
                            newRowDown += 1
                        if isBlockedDown:
                            count+=1

        else:
            if value == "A":
                isAmbulanceFound = True
    return count

#### Algorithm A heuristics

In [13]:
def h1A(node):
    return len(node["solutionPath"]) + h1Gbfs(node)

def h2A(node):
    return len(node["solutionPath"]) + h2Gbfs(node)

def h3A(node):
    return len(node["solutionPath"]) + h3Gbfs(node)

def h4A(node):
    return len(node["solutionPath"]) + h4Gbfs(node)

## 2.2 Input

In [14]:
def readInputFile(file_name):
    puzzleList = []
    input_file = open(file_name, 'r')
    lines = input_file.readlines()
    puzzleNumber = 1

    for line in lines:
        fuel_levels = {}
        puzzle = []
        line = line.strip();
        if line != "" and not line.startswith("#"):
            splitted_line = line.split(" ")
            puzzle_string = splitted_line[0]

            puzzle_row = []
            for i in range(len(puzzle_string)):
                if puzzle_string[i] not in fuel_levels and puzzle_string[i] != ".":
                    fuel_levels[puzzle_string[i]] = 100
                puzzle_row.append(puzzle_string[i])
                if len(puzzle_row) == 6:
                    puzzle.append(puzzle_row)
                    puzzle_row = []

            fuel_levels_array = splitted_line[1:]
            for fuel_level in fuel_levels_array:
                fuel_levels[fuel_level[0]] = int(fuel_level[1])
            puzzleList.append({
                "puzzle":puzzle,
                "fuel_levels":fuel_levels,
                "line":line,
                "puzzleNumber":puzzleNumber
            })
            puzzleNumber += 1
    return puzzleList

    input_file.close()

## 2.3 Output

### 2.3.1 Solution Files

In [15]:
def createSolFile(result, initialConfigString, algorithmType, puzzleNumber, fuelLevels):
    solFile = open("./output_files/"+algorithmType+"sol-"+str(puzzleNumber)+".txt", "w")
    if result["result"] == "ok":
        solFile.write("Initial board configuration: "+initialConfigString+"\n\n")

        splitted_line = initialConfigString.split(" ")
        puzzle_string = splitted_line[0]
        count = 0
        for i in range(len(puzzle_string)):
            solFile.write(puzzle_string[i]+" ")
            count+=1
            if count == 6:
                solFile.write("\n")
                count = 0
        solFile.write("\n")

        solFile.write("Car fuel available: ")
        fuelsString = ""
        for key in fuelLevels:
            fuelsString += key+":"+str(fuelLevels[key])+", "
        solFile.write(fuelsString[0: len(fuelsString) - 2]+"\n\n")

        runtimeVal = float("{:.2f}".format(result["runtime"]))
        solFile.write("Runtime: "+str(runtimeVal)+" seconds\n")

        solFile.write("Search path length: "+str(result["lengthSearchPaths"])+"\n")

        solFile.write("Solution path length: "+str(len(result["node"]["solutionPath"]))+"\n")

        solFile.write("Solution path: ")
        solPathString = ""
        for path in result["node"]["solutionPath"]:
            solPathString += path["carType"] +" "+ path["direction"] +" 1; "
        solFile.write(solPathString[0: len(solPathString) - 2])
        solFile.write("\n\n")

        for path in result["node"]["solutionPath"]:
            pathString = ""
            pathString += path["carType"] +" "+ path["direction"] +" 1       "
            pathString += str(path["fuelLevels"][path["carType"]])+" "
            for puzzleRow in path["puzzle"]:
                for puzzleElem in puzzleRow:
                    pathString += puzzleElem
            solFile.write(pathString+"\n")
        solFile.write("\n")

        finalPuzzle = result["node"]["puzzle"]
        for puzzleRow in finalPuzzle:
            rowString = ""
            for puzzleElem in puzzleRow:
                rowString += puzzleElem + " "
            solFile.write(rowString+"\n")

    else:
        solFile.write("Initial board configuration: "+initialConfigString+"\n\n")

        splitted_line = initialConfigString.split(" ")
        puzzle_string = splitted_line[0]
        count = 0
        for i in range(len(puzzle_string)):
            solFile.write(puzzle_string[i]+" ")
            count+=1
            if count == 6:
                solFile.write("\n")
                count = 0
        solFile.write("\n")

        solFile.write("Car fuel available: ")
        fuelsString = ""
        for key in fuelLevels:
            fuelsString += key+":"+str(fuelLevels[key])+", "
        solFile.write(fuelsString[0: len(fuelsString) - 2]+"\n\n")

        solFile.write("Sorry, could not solve the puzzle as specified.\n")

        solFile.write("Erorr: no solution found.\n\n")

        runtimeVal = float("{:.2f}".format(result["runtime"]))
        solFile.write("Runtime: "+str(runtimeVal)+" seconds")

    solFile.close()

### 2.3.2 Search Files

In [16]:
def createSearchFile(result, initialConfigString, algorithmType, puzzleNumber):
    searchFile = open("./output_files/"+algorithmType+"search-"+str(puzzleNumber)+".txt", "w")
    closedList = result["closedList"]
    for node in closedList:
        nodeString = ""
        if algorithmType.__contains__("ucs"):
            nodeString += "0 "
            nodeString += str(node["fOfN"]) + " 0 "
        elif algorithmType.__contains__("gbfs"):
            #h(n)
            nodeString += "0 0 " + str(node["fOfN"])+" "
        else:
            #f(n) g(n) h(n)
            hOfN = node["fOfN"] - len(node["solutionPath"])
            nodeString += str(node["fOfN"]) + " " + str(len(node["solutionPath"])) + " " + str(hOfN) + " "

        for puzzleRow in node["puzzle"]:
            for puzzleElem in puzzleRow:
                nodeString += puzzleElem
        nodeString += " "

        for key in node["fuelLevels"]:
            if node["fuelLevels"][key] != 100:
                nodeString += key + str(node["fuelLevels"][key]) + " "

        searchFile.write(nodeString+"\n")

    searchFile.close()

In [17]:
def solvePuzzle(puzzle, fuelLevels, initialStringConfig, puzzleNumber):
    print("Solving puzzle #"+str(puzzleNumber)+"\n")

    #UCS
    print("Using UCS")
    resultUCS = solver(puzzle, fuelLevels, hUcs, "ucs")
    createSolFile(resultUCS, initialStringConfig, "ucs-", puzzleNumber, fuelLevels)
    createSearchFile(resultUCS, initialStringConfig, "ucs-", puzzleNumber)
    print("\n")

    #GBFS heuristic 1
    print("Using GBFS heuristic 1")
    resultGBFS1 = solver(puzzle, fuelLevels, h1Gbfs, "gbfs")
    createSolFile(resultGBFS1, initialStringConfig, "gbfs-h1-", puzzleNumber, fuelLevels)
    createSearchFile(resultGBFS1, initialStringConfig, "gbfs-h1-", puzzleNumber)
    print("\n")

    #GBFS heuristic 2
    print("Using GBFS heuristic 2")
    resultGBFS2 = solver(puzzle, fuelLevels, h2Gbfs, "gbfs")
    createSolFile(resultGBFS2, initialStringConfig, "gbfs-h2-", puzzleNumber, fuelLevels)
    createSearchFile(resultGBFS2, initialStringConfig, "gbfs-h2-", puzzleNumber)
    print("\n")

    #GBFS heuristic 3
    print("Using GBFS heuristic 3")
    resultGBFS3 = solver(puzzle, fuelLevels, h3Gbfs, "gbfs")
    createSolFile(resultGBFS3, initialStringConfig, "gbfs-h3-", puzzleNumber, fuelLevels)
    createSearchFile(resultGBFS3, initialStringConfig, "gbfs-h3-", puzzleNumber)
    print("\n")

    #GBFS heuristic 4
    print("Using GBFS heuristic 4")
    resultGBFS4 = solver(puzzle, fuelLevels, h4Gbfs, "gbfs")
    createSolFile(resultGBFS4, initialStringConfig, "gbfs-h4-", puzzleNumber, fuelLevels)
    createSearchFile(resultGBFS4, initialStringConfig, "gbfs-h4-", puzzleNumber)
    print("\n")

    #A heuristic 1
    print("Using A heuristic 1")
    resultA1 = solver(puzzle, fuelLevels, h1A, "a")
    createSolFile(resultA1, initialStringConfig, "a-h1-", puzzleNumber, fuelLevels)
    createSearchFile(resultA1, initialStringConfig, "a-h1-", puzzleNumber)
    print("\n")

    #A heuristic 2
    print("Using A heuristic 2")
    resultA2 = solver(puzzle, fuelLevels, h2A, "a")
    createSolFile(resultA2, initialStringConfig, "a-h2-", puzzleNumber, fuelLevels)
    createSearchFile(resultA2, initialStringConfig, "a-h2-", puzzleNumber)
    print("\n")

    #A heuristic 3
    print("Using A heuristic 3")
    resultA3 = solver(puzzle, fuelLevels, h3A, "a")
    createSolFile(resultA3, initialStringConfig, "a-h3-", puzzleNumber, fuelLevels)
    createSearchFile(resultA3, initialStringConfig, "a-h3-", puzzleNumber)
    print("\n")

    #A heuristic 4
    print("Using A heuristic 4")
    resultA4 = solver(puzzle, fuelLevels, h4A, "a")
    createSolFile(resultA4, initialStringConfig, "a-h4-", puzzleNumber, fuelLevels)
    createSearchFile(resultA4, initialStringConfig, "a-h4-", puzzleNumber)
    print("\n")

puzzleList = readInputFile('./Sample/sample-input.txt')
for puzzleElem in puzzleList:
    solvePuzzle(puzzleElem["puzzle"], puzzleElem["fuel_levels"], puzzleElem["line"], puzzleElem["puzzleNumber"])

Solving puzzle #1

Using UCS
Length closedList: 6; Length open list: 12; Closed similar:7; Open similar:10

Using GBFS heuristic 1
Length closedList: 3; Length open list: 11; Closed similar:3; Open similar:1

Using GBFS heuristic 2
Length closedList: 3; Length open list: 11; Closed similar:3; Open similar:1

Using GBFS heuristic 3
Length closedList: 3; Length open list: 11; Closed similar:3; Open similar:1

Using GBFS heuristic 4
Length closedList: 3; Length open list: 11; Closed similar:3; Open similar:1

Using A heuristic 1
Length closedList: 6; Length open list: 12; Closed similar:7; Open similar:10

Using A heuristic 2
Length closedList: 6; Length open list: 12; Closed similar:7; Open similar:10

Using A heuristic 3
Length closedList: 3; Length open list: 11; Closed similar:3; Open similar:1

Using A heuristic 4
Length closedList: 6; Length open list: 12; Closed similar:7; Open similar:10

Solving puzzle #2

Using UCS
Length closedList: 1835; Length open list: 1751; Closed similar:

# 3. Analysis

### Reading the file with 50 random puzzles

In [18]:
import xlsxwriter
puzzle50RandomList = readInputFile('50randomPuzzles.txt')

workbook = xlsxwriter.Workbook('50puzzlesStatistics.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write(0, 0, "Puzzle Number")
worksheet.write(0, 1, "Algorithm")
worksheet.write(0, 2, "Heuristic")
worksheet.write(0, 3, "Length of the Solution")
worksheet.write(0, 4, "Length of the Search Path")
worksheet.write(0, 5, "Execution Time (in seconds)")

def solvePuzzleInSpreadsheet(puzzle, fuelLevels, puzzleNumber, worksheet):

    print("Solving puzzle #"+str(puzzleNumber)+"\n")

    #UCS
    worksheet.write(((puzzleNumber - 1) * 9)+1, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+1, 1, "UCS")
    worksheet.write(((puzzleNumber - 1) * 9)+1, 2, "NA")
    print("Using UCS")
    resultUCS = solver(puzzle, fuelLevels, hUcs, "ucs")
    if resultUCS["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+1, 3, len(resultUCS["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+1, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+1, 4, resultUCS["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+1, 5, float("{:.2f}".format(resultUCS["runtime"])))
    print("\n")

    #GBFS heuristic 1
    worksheet.write(((puzzleNumber - 1) * 9)+2, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+2, 1, "GBFS")
    worksheet.write(((puzzleNumber - 1) * 9)+2, 2, "h1")
    print("Using GBFS heuristic 1")
    resultGBFS1 = solver(puzzle, fuelLevels, h1Gbfs, "gbfs")

    if resultGBFS1["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+2, 3, len(resultGBFS1["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+2, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+2, 4, resultGBFS1["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+2, 5, float("{:.2f}".format(resultGBFS1["runtime"])))
    print("\n")

    #GBFS heuristic 2
    worksheet.write(((puzzleNumber - 1) * 9)+3, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+3, 1, "GBFS")
    worksheet.write(((puzzleNumber - 1) * 9)+3, 2, "h2")
    print("Using GBFS heuristic 2")
    resultGBFS2 = solver(puzzle, fuelLevels, h2Gbfs, "gbfs")

    if resultGBFS2["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+3, 3, len(resultGBFS2["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+3, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+3, 4, resultGBFS2["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+3, 5, float("{:.2f}".format(resultGBFS2["runtime"])))
    print("\n")

    #GBFS heuristic 3
    worksheet.write(((puzzleNumber - 1) * 9)+4, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+4, 1, "GBFS")
    worksheet.write(((puzzleNumber - 1) * 9)+4, 2, "h3")
    print("Using GBFS heuristic 3")
    resultGBFS3 = solver(puzzle, fuelLevels, h3Gbfs, "gbfs")

    if resultGBFS3["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+4, 3, len(resultGBFS3["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+4, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+4, 4, resultGBFS3["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+4, 5, float("{:.2f}".format(resultGBFS3["runtime"])))
    print("\n")

    #GBFS heuristic 4
    worksheet.write(((puzzleNumber - 1) * 9)+5, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+5, 1, "GBFS")
    worksheet.write(((puzzleNumber - 1) * 9)+5, 2, "h4")
    print("Using GBFS heuristic 4")
    resultGBFS4 = solver(puzzle, fuelLevels, h4Gbfs, "gbfs")

    if resultGBFS4["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+5, 3, len(resultGBFS4["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+5, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+5, 4, resultGBFS4["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+5, 5, float("{:.2f}".format(resultGBFS4["runtime"])))
    print("\n")

    #A heuristic 1
    worksheet.write(((puzzleNumber - 1) * 9)+6, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+6, 1, "A/A*")
    worksheet.write(((puzzleNumber - 1) * 9)+6, 2, "h1")
    print("Using A heuristic 1")
    resultA1 = solver(puzzle, fuelLevels, h1A, "a")

    if resultA1["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+6, 3, len(resultA1["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+6, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+6, 4, resultA1["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+6, 5, float("{:.2f}".format(resultA1["runtime"])))
    print("\n")

    #A heuristic 2
    worksheet.write(((puzzleNumber - 1) * 9)+7, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+7, 1, "A/A*")
    worksheet.write(((puzzleNumber - 1) * 9)+7, 2, "h2")
    print("Using A heuristic 2")
    resultA2 = solver(puzzle, fuelLevels, h2A, "a")

    if resultA2["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+7, 3, len(resultA2["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+7, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+7, 4, resultA2["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+7, 5, float("{:.2f}".format(resultA2["runtime"])))
    print("\n")

    #A heuristic 3
    worksheet.write(((puzzleNumber - 1) * 9)+8, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+8, 1, "A/A*")
    worksheet.write(((puzzleNumber - 1) * 9)+8, 2, "h3")
    print("Using A heuristic 3")
    resultA3 = solver(puzzle, fuelLevels, h3A, "a")

    if resultA3["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+8, 3, len(resultA3["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+8, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+8, 4, resultA3["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+8, 5, float("{:.2f}".format(resultA3["runtime"])))
    print("\n")

    #A heuristic 4
    worksheet.write(((puzzleNumber - 1) * 9)+9, 0, puzzleNumber)
    worksheet.write(((puzzleNumber - 1) * 9)+9, 1, "A/A*")
    worksheet.write(((puzzleNumber - 1) * 9)+9, 2, "h4")
    print("Using A heuristic 4")
    resultA4 = solver(puzzle, fuelLevels, h4A, "a")

    if resultA4["result"] == "ok":
        worksheet.write(((puzzleNumber - 1) * 9)+9, 3, len(resultA4["node"]["solutionPath"]))
    else:
        worksheet.write(((puzzleNumber - 1) * 9)+9, 3, "No solution!")

    worksheet.write(((puzzleNumber - 1) * 9)+9, 4, resultA4["lengthSearchPaths"])
    worksheet.write(((puzzleNumber - 1) * 9)+9, 5, float("{:.2f}".format(resultA4["runtime"])))
    print("\n")

for puzzleElem in puzzle50RandomList:
    solvePuzzleInSpreadsheet(puzzleElem["puzzle"], puzzleElem["fuel_levels"], puzzleElem["puzzleNumber"], worksheet)

workbook.close()

Solving puzzle #1

Using UCS
Length closedList: 1505; Length open list: 227; Closed similar:8031; Open similar:7025

Using GBFS heuristic 1
Length closedList: 228; Length open list: 316; Closed similar:838; Open similar:853

Using GBFS heuristic 2
Length closedList: 228; Length open list: 316; Closed similar:838; Open similar:853

Using GBFS heuristic 3
Length closedList: 228; Length open list: 316; Closed similar:838; Open similar:853

Using GBFS heuristic 4
Length closedList: 228; Length open list: 316; Closed similar:838; Open similar:853

Using A heuristic 1
Length closedList: 1199; Length open list: 432; Closed similar:5793; Open similar:5666

Using A heuristic 2
Length closedList: 1199; Length open list: 432; Closed similar:5793; Open similar:5666

Using A heuristic 3
Length closedList: 256; Length open list: 337; Closed similar:925; Open similar:990

Using A heuristic 4
Length closedList: 1199; Length open list: 432; Closed similar:5793; Open similar:5666

Solving puzzle #2

Usi

### Analyzing solution lengths

In [19]:
import pandas as pd

df = pd.read_excel('50puzzlesStatistics.xlsx')

h1SolLengthSum = 0
h1SolLengthCount = 0

h2SolLengthSum = 0
h2SolLengthCount = 0

h3SolLengthSum = 0
h3SolLengthCount = 0

h4SolLengthSum = 0
h4SolLengthCount = 0

ucsSolLengthSum = 0
ucsSolLengthCount = 0

gbfsSolLengthSum = 0
gbfsSolLengthCount = 0

aSolLengthSum = 0
aSolLengthCount = 0


for index, row in df.iterrows():
    if row["Length of the Solution"] != "No solution!":
        if row["Algorithm"] == "UCS":
            ucsSolLengthCount += 1
            ucsSolLengthSum += row["Length of the Solution"]
        elif row["Algorithm"] == "GBFS":
            gbfsSolLengthCount += 1
            gbfsSolLengthSum += row["Length of the Solution"]
        elif row["Algorithm"] == "A/A*":
            aSolLengthCount += 1
            aSolLengthSum += row["Length of the Solution"]

        if row["Heuristic"] == "h1":
            h1SolLengthCount += 1
            h1SolLengthSum += row["Length of the Solution"]
        elif row["Heuristic"] == "h2":
            h2SolLengthCount += 1
            h2SolLengthSum += row["Length of the Solution"]
        elif row["Heuristic"] == "h3":
            h3SolLengthCount += 1
            h3SolLengthSum += row["Length of the Solution"]
        elif row["Heuristic"] == "h4":
            h4SolLengthCount += 1
            h4SolLengthSum += row["Length of the Solution"]

print("Avg solution length H1: "+str(h1SolLengthSum / h1SolLengthCount)+"\n");
print("Avg solution length H2: "+str(h2SolLengthSum / h2SolLengthCount)+"\n");
print("Avg solution length H3: "+str(h3SolLengthSum / h3SolLengthCount)+"\n");
print("Avg solution length H4: "+str(h4SolLengthSum / h4SolLengthCount)+"\n");

print("Avg solution length UCS: "+str(ucsSolLengthSum / ucsSolLengthCount)+"\n");
print("Avg solution length GBFS: "+str(gbfsSolLengthSum / gbfsSolLengthCount)+"\n");
print("Avg solution length A/A*: "+str(aSolLengthSum / aSolLengthCount)+"\n");

Avg solution length H1: 9.060975609756097

Avg solution length H2: 9.060975609756097

Avg solution length H3: 9.317073170731707

Avg solution length H4: 8.975609756097562

Avg solution length UCS: 8.536585365853659

Avg solution length GBFS: 9.542682926829269

Avg solution length A/A*: 8.664634146341463



### Analyzing execution times

In [20]:
h1RuntimeSum = 0
h1RuntimeCount = 0

h2RuntimeSum = 0
h2RuntimeCount = 0

h3RuntimeSum = 0
h3RuntimeCount = 0

h4RuntimeSum = 0
h4RuntimeCount = 0

ucsRuntimeSum = 0
ucsRuntimeCount = 0

gbfsRuntimeSum = 0
gbfsRuntimeCount = 0

aRuntimeSum = 0
aRuntimeCount = 0


for index, row in df.iterrows():
    if row["Length of the Solution"] != "No solution!":
        if row["Algorithm"] == "UCS":
            ucsRuntimeCount += 1
            ucsRuntimeSum += row["Execution Time (in seconds)"]
        elif row["Algorithm"] == "GBFS":
            gbfsRuntimeCount += 1
            gbfsRuntimeSum += row["Execution Time (in seconds)"]
        elif row["Algorithm"] == "A/A*":
            aRuntimeCount += 1
            aRuntimeSum += row["Execution Time (in seconds)"]

        if row["Heuristic"] == "h1":
            h1RuntimeCount += 1
            h1RuntimeSum += row["Execution Time (in seconds)"]
        elif row["Heuristic"] == "h2":
            h2RuntimeCount += 1
            h2RuntimeSum += row["Execution Time (in seconds)"]
        elif row["Heuristic"] == "h3":
            h3RuntimeCount += 1
            h3RuntimeSum += row["Execution Time (in seconds)"]
        elif row["Heuristic"] == "h4":
            h4RuntimeCount += 1
            h4RuntimeSum += row["Execution Time (in seconds)"]

print("Avg execution time H1: "+str(h1RuntimeSum / h1RuntimeCount)+"\n");
print("Avg execution time H2: "+str(h2RuntimeSum / h2RuntimeCount)+"\n");
print("Avg execution time H3: "+str(h3RuntimeSum / h3RuntimeCount)+"\n");
print("Avg execution time H4: "+str(h4RuntimeSum / h4RuntimeCount)+"\n");

print("Avg execution time UCS: "+str(ucsRuntimeSum / ucsRuntimeCount)+"\n");
print("Avg execution time GBFS: "+str(gbfsRuntimeSum / gbfsRuntimeCount)+"\n");
print("Avg execution time A/A*: "+str(aRuntimeSum / aRuntimeCount)+"\n");

Avg execution time H1: 6.2387804878048785

Avg execution time H2: 6.510487804878049

Avg execution time H3: 4.413902439024389

Avg execution time H4: 5.872560975609756

Avg execution time UCS: 12.512926829268292

Avg execution time GBFS: 3.870670731707314

Avg execution time A/A*: 7.6471951219512135

